In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
import featuretools as ft

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

### Visualization

## Data Cleaning

In [3]:
train_df["previous_year_rating"].value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
Name: previous_year_rating, dtype: int64

In [4]:
train_df["previous_year_rating"].fillna(-999, inplace=True)
test_df["previous_year_rating"].fillna(-999, inplace=True)

In [5]:
# OHE

In [6]:
train_df["reg_num"] = train_df["region"].str.extract("(\d+)")
train_df["reg_num"] = train_df["reg_num"].astype(int)

test_df["reg_num"] = test_df["region"].str.extract("(\d+)")
test_df["reg_num"] = test_df["reg_num"].astype(int)

In [7]:
train_df = pd.concat([train_df, pd.get_dummies(train_df["department"], prefix="dep_")], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["department"], prefix="dep_")], axis=1)

In [8]:
train_df = pd.concat([train_df, pd.get_dummies(train_df["gender"], prefix="gen_")], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["gender"], prefix="gen_")], axis=1)

In [9]:
train_df = pd.concat([train_df, pd.get_dummies(train_df["recruitment_channel"], prefix="rec_")], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["recruitment_channel"], prefix="rec_")], axis=1)

In [10]:
train_df["education"] = train_df["education"].map({"Bachelor's": 2, "Master's & above": 4, "Below Secondary": 1})
test_df["education"] = test_df["education"].map({"Bachelor's": 2, "Master's & above": 4, "Below Secondary": 1})

In [11]:
train_df["education"].fillna(-999, inplace=True)
test_df["education"].fillna(-999, inplace=True)

In [12]:
#Dropping columns

In [13]:
emp_id = test_df["employee_id"]
train_df.drop(["department", "region", "gender", "recruitment_channel", "employee_id"], axis=1, inplace=True)
test_df.drop(["department", "region", "gender", "recruitment_channel", "employee_id"], axis=1, inplace=True)

## Feature Engineering

In [14]:
# binning age
bin_ranges = [19, 25, 30, 35, 40, 45, 50, 55, 60, 65]
bin_names = [1, 2, 3, 4, 5, 6, 7, 8, 9]

train_df['Age_bin_custom_label'] = pd.cut(
                                           np.array(
                                              train_df['age']), 
                                              bins=bin_ranges,            
                                              labels=bin_names)

test_df['Age_bin_custom_label'] = pd.cut(
                                           np.array(
                                              test_df['age']), 
                                              bins=bin_ranges,            
                                              labels=bin_names)

In [15]:
train_df['Age_bin_custom_label'] = train_df['Age_bin_custom_label'].astype(int)
test_df['Age_bin_custom_label'] = test_df['Age_bin_custom_label'].astype(int)

In [16]:
# transforming age
train_df['Age_log'] = np.log((1+ train_df['age']))
test_df['Age_log'] = np.log((1+ test_df['age']))

In [17]:
# transforming los
train_df["los_log"] = np.log((1+train_df["length_of_service"]))
test_df["los_log"] = np.log((1+test_df["length_of_service"]))

In [18]:
# transforming training_score
train_df["training_log"] = np.log((1+train_df["avg_training_score"]))
test_df["training_log"] = np.log((1+test_df["avg_training_score"]))

In [19]:
train_df.drop(["age", "length_of_service", "avg_training_score"], axis=1, inplace=True)
test_df.drop(["age", "length_of_service", "avg_training_score"], axis=1, inplace=True)

## Data Prep

In [20]:
y_train = train_df["is_promoted"]
X_train = train_df.drop("is_promoted", axis=1)

X_test = test_df

# CV

In [22]:
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import GridSearchCV

In [22]:
# cb - cv
kf = KFold(n_splits=5)
cb_model = cb.CatBoostClassifier(n_estimators=1000, max_depth = 8, random_state=7, scale_pos_weight=3, thread_count=2)
scores = cross_val_score(cb_model, X_train, y_train, scoring="f1", cv=kf)
x = scores.mean()
x

Learning rate set to 0.048842
0:	learn: 0.6431830	total: 157ms	remaining: 2m 37s
1:	learn: 0.6048274	total: 270ms	remaining: 2m 14s
2:	learn: 0.5645729	total: 369ms	remaining: 2m 2s
3:	learn: 0.5438784	total: 463ms	remaining: 1m 55s
4:	learn: 0.5233744	total: 564ms	remaining: 1m 52s
5:	learn: 0.4979182	total: 667ms	remaining: 1m 50s
6:	learn: 0.4779855	total: 758ms	remaining: 1m 47s
7:	learn: 0.4587838	total: 845ms	remaining: 1m 44s
8:	learn: 0.4491547	total: 946ms	remaining: 1m 44s
9:	learn: 0.4425276	total: 1.01s	remaining: 1m 40s
10:	learn: 0.4341164	total: 1.09s	remaining: 1m 38s
11:	learn: 0.4254399	total: 1.19s	remaining: 1m 37s
12:	learn: 0.4082006	total: 1.28s	remaining: 1m 37s
13:	learn: 0.4012909	total: 1.38s	remaining: 1m 36s
14:	learn: 0.3921540	total: 1.47s	remaining: 1m 36s
15:	learn: 0.3896663	total: 1.55s	remaining: 1m 35s
16:	learn: 0.3866578	total: 1.6s	remaining: 1m 32s
17:	learn: 0.3795824	total: 1.72s	remaining: 1m 33s
18:	learn: 0.3731510	total: 1.79s	remaining: 1

157:	learn: 0.2730327	total: 13.3s	remaining: 1m 11s
158:	learn: 0.2730012	total: 13.4s	remaining: 1m 10s
159:	learn: 0.2728876	total: 13.5s	remaining: 1m 10s
160:	learn: 0.2728872	total: 13.5s	remaining: 1m 10s
161:	learn: 0.2728314	total: 13.6s	remaining: 1m 10s
162:	learn: 0.2724590	total: 13.7s	remaining: 1m 10s
163:	learn: 0.2724088	total: 13.8s	remaining: 1m 10s
164:	learn: 0.2724004	total: 13.8s	remaining: 1m 9s
165:	learn: 0.2722811	total: 13.9s	remaining: 1m 9s
166:	learn: 0.2722800	total: 14s	remaining: 1m 9s
167:	learn: 0.2721053	total: 14s	remaining: 1m 9s
168:	learn: 0.2721041	total: 14.1s	remaining: 1m 9s
169:	learn: 0.2720976	total: 14.1s	remaining: 1m 9s
170:	learn: 0.2720503	total: 14.2s	remaining: 1m 8s
171:	learn: 0.2719223	total: 14.3s	remaining: 1m 8s
172:	learn: 0.2718397	total: 14.4s	remaining: 1m 8s
173:	learn: 0.2717896	total: 14.4s	remaining: 1m 8s
174:	learn: 0.2717270	total: 14.5s	remaining: 1m 8s
175:	learn: 0.2717124	total: 14.6s	remaining: 1m 8s
176:	lear

316:	learn: 0.2644997	total: 24.5s	remaining: 52.9s
317:	learn: 0.2644528	total: 24.6s	remaining: 52.8s
318:	learn: 0.2643752	total: 24.7s	remaining: 52.8s
319:	learn: 0.2643244	total: 24.8s	remaining: 52.7s
320:	learn: 0.2641697	total: 24.9s	remaining: 52.7s
321:	learn: 0.2641697	total: 25s	remaining: 52.5s
322:	learn: 0.2641610	total: 25s	remaining: 52.5s
323:	learn: 0.2641600	total: 25.1s	remaining: 52.3s
324:	learn: 0.2640218	total: 25.2s	remaining: 52.3s
325:	learn: 0.2640033	total: 25.2s	remaining: 52.2s
326:	learn: 0.2640032	total: 25.3s	remaining: 52.1s
327:	learn: 0.2640012	total: 25.4s	remaining: 52s
328:	learn: 0.2640009	total: 25.4s	remaining: 51.9s
329:	learn: 0.2640006	total: 25.5s	remaining: 51.8s
330:	learn: 0.2639236	total: 25.6s	remaining: 51.7s
331:	learn: 0.2639235	total: 25.6s	remaining: 51.6s
332:	learn: 0.2639232	total: 25.7s	remaining: 51.5s
333:	learn: 0.2638889	total: 25.8s	remaining: 51.4s
334:	learn: 0.2637653	total: 25.9s	remaining: 51.4s
335:	learn: 0.2637

477:	learn: 0.2591974	total: 37.1s	remaining: 40.5s
478:	learn: 0.2591912	total: 37.2s	remaining: 40.5s
479:	learn: 0.2591912	total: 37.3s	remaining: 40.4s
480:	learn: 0.2590890	total: 37.4s	remaining: 40.3s
481:	learn: 0.2589440	total: 37.5s	remaining: 40.3s
482:	learn: 0.2589439	total: 37.5s	remaining: 40.2s
483:	learn: 0.2589395	total: 37.6s	remaining: 40.1s
484:	learn: 0.2589395	total: 37.7s	remaining: 40s
485:	learn: 0.2589395	total: 37.7s	remaining: 39.9s
486:	learn: 0.2588688	total: 37.8s	remaining: 39.8s
487:	learn: 0.2588559	total: 37.9s	remaining: 39.8s
488:	learn: 0.2588344	total: 38s	remaining: 39.7s
489:	learn: 0.2588342	total: 38.1s	remaining: 39.6s
490:	learn: 0.2588339	total: 38.1s	remaining: 39.5s
491:	learn: 0.2588338	total: 38.2s	remaining: 39.4s
492:	learn: 0.2588309	total: 38.3s	remaining: 39.3s
493:	learn: 0.2586531	total: 38.4s	remaining: 39.3s
494:	learn: 0.2586525	total: 38.4s	remaining: 39.2s
495:	learn: 0.2586524	total: 38.5s	remaining: 39.1s
496:	learn: 0.25

638:	learn: 0.2556135	total: 50.2s	remaining: 28.4s
639:	learn: 0.2556048	total: 50.3s	remaining: 28.3s
640:	learn: 0.2554431	total: 50.4s	remaining: 28.2s
641:	learn: 0.2554398	total: 50.5s	remaining: 28.2s
642:	learn: 0.2554232	total: 50.6s	remaining: 28.1s
643:	learn: 0.2553875	total: 50.7s	remaining: 28s
644:	learn: 0.2553531	total: 50.8s	remaining: 27.9s
645:	learn: 0.2553340	total: 50.9s	remaining: 27.9s
646:	learn: 0.2553337	total: 51s	remaining: 27.8s
647:	learn: 0.2553296	total: 51s	remaining: 27.7s
648:	learn: 0.2553293	total: 51.1s	remaining: 27.6s
649:	learn: 0.2552559	total: 51.2s	remaining: 27.6s
650:	learn: 0.2552380	total: 51.3s	remaining: 27.5s
651:	learn: 0.2552124	total: 51.3s	remaining: 27.4s
652:	learn: 0.2552108	total: 51.4s	remaining: 27.3s
653:	learn: 0.2551845	total: 51.5s	remaining: 27.3s
654:	learn: 0.2551843	total: 51.6s	remaining: 27.2s
655:	learn: 0.2551831	total: 51.6s	remaining: 27.1s
656:	learn: 0.2551830	total: 51.7s	remaining: 27s
657:	learn: 0.255173

...

462:	learn: 0.2593334	total: 36s	remaining: 41.8s
463:	learn: 0.2593331	total: 36.1s	remaining: 41.7s
464:	learn: 0.2593330	total: 36.1s	remaining: 41.6s
465:	learn: 0.2593152	total: 36.2s	remaining: 41.5s
466:	learn: 0.2592828	total: 36.3s	remaining: 41.5s
467:	learn: 0.2592784	total: 36.4s	remaining: 41.4s
468:	learn: 0.2592483	total: 36.5s	remaining: 41.3s
469:	learn: 0.2592483	total: 36.6s	remaining: 41.2s
470:	learn: 0.2592020	total: 36.7s	remaining: 41.2s
471:	learn: 0.2591975	total: 36.8s	remaining: 41.1s
472:	learn: 0.2591449	total: 36.9s	remaining: 41.1s
473:	learn: 0.2590740	total: 37s	remaining: 41s
474:	learn: 0.2590645	total: 37s	remaining: 40.9s
475:	learn: 0.2590645	total: 37.1s	remaining: 40.8s
476:	learn: 0.2589701	total: 37.2s	remaining: 40.8s
477:	learn: 0.2589694	total: 37.2s	remaining: 40.7s
478:	learn: 0.2589694	total: 37.3s	remaining: 40.6s
479:	learn: 0.2589693	total: 37.4s	remaining: 40.5s
480:	learn: 0.2589693	total: 37.4s	remaining: 40.4s
481:	learn: 0.258854

623:	learn: 0.2559345	total: 49.6s	remaining: 29.9s
624:	learn: 0.2558971	total: 49.7s	remaining: 29.8s
625:	learn: 0.2558957	total: 49.8s	remaining: 29.7s
626:	learn: 0.2558956	total: 49.8s	remaining: 29.6s
627:	learn: 0.2558376	total: 49.9s	remaining: 29.6s
628:	learn: 0.2558376	total: 50s	remaining: 29.5s
629:	learn: 0.2558376	total: 50.1s	remaining: 29.4s
630:	learn: 0.2558298	total: 50.2s	remaining: 29.3s
631:	learn: 0.2558296	total: 50.2s	remaining: 29.2s
632:	learn: 0.2558189	total: 50.3s	remaining: 29.2s
633:	learn: 0.2558180	total: 50.4s	remaining: 29.1s
634:	learn: 0.2558178	total: 50.5s	remaining: 29s
635:	learn: 0.2558175	total: 50.5s	remaining: 28.9s
636:	learn: 0.2558105	total: 50.6s	remaining: 28.9s
637:	learn: 0.2558030	total: 50.7s	remaining: 28.8s
638:	learn: 0.2557809	total: 50.8s	remaining: 28.7s
639:	learn: 0.2557807	total: 50.9s	remaining: 28.6s
640:	learn: 0.2557801	total: 51s	remaining: 28.5s
641:	learn: 0.2556636	total: 51.1s	remaining: 28.5s
642:	learn: 0.2554

785:	learn: 0.2539241	total: 1m 2s	remaining: 17.1s
786:	learn: 0.2539210	total: 1m 2s	remaining: 17s
787:	learn: 0.2538385	total: 1m 2s	remaining: 16.9s
788:	learn: 0.2538278	total: 1m 3s	remaining: 16.9s
789:	learn: 0.2538191	total: 1m 3s	remaining: 16.8s
790:	learn: 0.2538180	total: 1m 3s	remaining: 16.7s
791:	learn: 0.2537967	total: 1m 3s	remaining: 16.6s
792:	learn: 0.2537712	total: 1m 3s	remaining: 16.5s
793:	learn: 0.2537449	total: 1m 3s	remaining: 16.5s
794:	learn: 0.2537222	total: 1m 3s	remaining: 16.4s
795:	learn: 0.2536929	total: 1m 3s	remaining: 16.3s
796:	learn: 0.2536797	total: 1m 3s	remaining: 16.2s
797:	learn: 0.2536660	total: 1m 3s	remaining: 16.2s
798:	learn: 0.2536565	total: 1m 3s	remaining: 16.1s
799:	learn: 0.2535948	total: 1m 4s	remaining: 16s
800:	learn: 0.2535948	total: 1m 4s	remaining: 15.9s
801:	learn: 0.2535880	total: 1m 4s	remaining: 15.9s
802:	learn: 0.2535878	total: 1m 4s	remaining: 15.8s
803:	learn: 0.2535878	total: 1m 4s	remaining: 15.7s
804:	learn: 0.25

942:	learn: 0.2518106	total: 1m 16s	remaining: 4.61s
943:	learn: 0.2518089	total: 1m 16s	remaining: 4.52s
944:	learn: 0.2518084	total: 1m 16s	remaining: 4.44s
945:	learn: 0.2517759	total: 1m 16s	remaining: 4.37s
946:	learn: 0.2517759	total: 1m 16s	remaining: 4.28s
947:	learn: 0.2517757	total: 1m 16s	remaining: 4.2s
948:	learn: 0.2517757	total: 1m 16s	remaining: 4.12s
949:	learn: 0.2517699	total: 1m 16s	remaining: 4.04s
950:	learn: 0.2517683	total: 1m 16s	remaining: 3.96s
951:	learn: 0.2517680	total: 1m 16s	remaining: 3.88s
952:	learn: 0.2517678	total: 1m 17s	remaining: 3.8s
953:	learn: 0.2517642	total: 1m 17s	remaining: 3.72s
954:	learn: 0.2516929	total: 1m 17s	remaining: 3.64s
955:	learn: 0.2516928	total: 1m 17s	remaining: 3.55s
956:	learn: 0.2516928	total: 1m 17s	remaining: 3.47s
957:	learn: 0.2516928	total: 1m 17s	remaining: 3.39s
958:	learn: 0.2516841	total: 1m 17s	remaining: 3.31s
959:	learn: 0.2516841	total: 1m 17s	remaining: 3.23s
960:	learn: 0.2516841	total: 1m 17s	remaining: 3

0.52267242517532142

In [23]:
scores

array([ 0.53939782,  0.5       ,  0.52336449,  0.53164557,  0.51895425])

## Model

In [25]:
model = cb.CatBoostClassifier(n_estimators=1000, max_depth = 5, random_state=7, scale_pos_weight=3, thread_count=2)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Learning rate set to 0.052026
0:	learn: 0.6468988	total: 27.7ms	remaining: 27.7s
1:	learn: 0.6012804	total: 53.5ms	remaining: 26.7s
2:	learn: 0.5672967	total: 81.8ms	remaining: 27.2s
3:	learn: 0.5425937	total: 107ms	remaining: 26.6s
4:	learn: 0.5201368	total: 130ms	remaining: 25.9s
5:	learn: 0.5011014	total: 155ms	remaining: 25.7s
6:	learn: 0.4831795	total: 189ms	remaining: 26.8s
7:	learn: 0.4718245	total: 213ms	remaining: 26.4s
8:	learn: 0.4595296	total: 239ms	remaining: 26.3s
9:	learn: 0.4475755	total: 265ms	remaining: 26.3s
10:	learn: 0.4392361	total: 288ms	remaining: 25.9s
11:	learn: 0.4314679	total: 320ms	remaining: 26.4s
12:	learn: 0.4248727	total: 349ms	remaining: 26.5s
13:	learn: 0.4186846	total: 392ms	remaining: 27.6s
14:	learn: 0.4130963	total: 420ms	remaining: 27.6s
15:	learn: 0.4087325	total: 446ms	remaining: 27.4s
16:	learn: 0.4052338	total: 472ms	remaining: 27.3s
17:	learn: 0.4023521	total: 501ms	remaining: 27.3s
18:	learn: 0.3907769	total: 533ms	remaining: 27.5s
19:	lear

162:	learn: 0.2795200	total: 4.76s	remaining: 24.5s
163:	learn: 0.2794471	total: 4.79s	remaining: 24.4s
164:	learn: 0.2793693	total: 4.82s	remaining: 24.4s
165:	learn: 0.2792042	total: 4.84s	remaining: 24.3s
166:	learn: 0.2791113	total: 4.86s	remaining: 24.3s
167:	learn: 0.2790188	total: 4.89s	remaining: 24.2s
168:	learn: 0.2789218	total: 4.91s	remaining: 24.2s
169:	learn: 0.2788052	total: 4.94s	remaining: 24.1s
170:	learn: 0.2786983	total: 4.96s	remaining: 24.1s
171:	learn: 0.2786342	total: 5s	remaining: 24.1s
172:	learn: 0.2785652	total: 5.02s	remaining: 24s
173:	learn: 0.2783419	total: 5.05s	remaining: 24s
174:	learn: 0.2782386	total: 5.07s	remaining: 23.9s
175:	learn: 0.2781407	total: 5.1s	remaining: 23.9s
176:	learn: 0.2780972	total: 5.12s	remaining: 23.8s
177:	learn: 0.2780224	total: 5.15s	remaining: 23.8s
178:	learn: 0.2779214	total: 5.17s	remaining: 23.7s
179:	learn: 0.2777923	total: 5.2s	remaining: 23.7s
180:	learn: 0.2776608	total: 5.22s	remaining: 23.6s
181:	learn: 0.2775844

324:	learn: 0.2673404	total: 8.99s	remaining: 18.7s
325:	learn: 0.2672911	total: 9.02s	remaining: 18.6s
326:	learn: 0.2672584	total: 9.04s	remaining: 18.6s
327:	learn: 0.2671524	total: 9.07s	remaining: 18.6s
328:	learn: 0.2671218	total: 9.1s	remaining: 18.6s
329:	learn: 0.2670784	total: 9.12s	remaining: 18.5s
330:	learn: 0.2669970	total: 9.15s	remaining: 18.5s
331:	learn: 0.2668092	total: 9.17s	remaining: 18.5s
332:	learn: 0.2667652	total: 9.2s	remaining: 18.4s
333:	learn: 0.2667275	total: 9.23s	remaining: 18.4s
334:	learn: 0.2666754	total: 9.26s	remaining: 18.4s
335:	learn: 0.2663532	total: 9.28s	remaining: 18.3s
336:	learn: 0.2663268	total: 9.31s	remaining: 18.3s
337:	learn: 0.2663008	total: 9.33s	remaining: 18.3s
338:	learn: 0.2662347	total: 9.36s	remaining: 18.3s
339:	learn: 0.2660369	total: 9.38s	remaining: 18.2s
340:	learn: 0.2659718	total: 9.41s	remaining: 18.2s
341:	learn: 0.2659308	total: 9.45s	remaining: 18.2s
342:	learn: 0.2658958	total: 9.47s	remaining: 18.1s
343:	learn: 0.

484:	learn: 0.2594486	total: 13.1s	remaining: 14s
485:	learn: 0.2594052	total: 13.2s	remaining: 13.9s
486:	learn: 0.2593873	total: 13.2s	remaining: 13.9s
487:	learn: 0.2593381	total: 13.2s	remaining: 13.9s
488:	learn: 0.2593078	total: 13.3s	remaining: 13.8s
489:	learn: 0.2592659	total: 13.3s	remaining: 13.8s
490:	learn: 0.2592405	total: 13.3s	remaining: 13.8s
491:	learn: 0.2592051	total: 13.3s	remaining: 13.8s
492:	learn: 0.2591885	total: 13.4s	remaining: 13.7s
493:	learn: 0.2591292	total: 13.4s	remaining: 13.7s
494:	learn: 0.2590610	total: 13.4s	remaining: 13.7s
495:	learn: 0.2590313	total: 13.4s	remaining: 13.7s
496:	learn: 0.2589938	total: 13.5s	remaining: 13.6s
497:	learn: 0.2589403	total: 13.5s	remaining: 13.6s
498:	learn: 0.2589180	total: 13.5s	remaining: 13.6s
499:	learn: 0.2588565	total: 13.5s	remaining: 13.5s
500:	learn: 0.2588243	total: 13.6s	remaining: 13.5s
501:	learn: 0.2588019	total: 13.6s	remaining: 13.5s
502:	learn: 0.2587245	total: 13.6s	remaining: 13.5s
503:	learn: 0.

645:	learn: 0.2534245	total: 17.4s	remaining: 9.52s
646:	learn: 0.2533952	total: 17.4s	remaining: 9.49s
647:	learn: 0.2533737	total: 17.4s	remaining: 9.47s
648:	learn: 0.2533462	total: 17.5s	remaining: 9.44s
649:	learn: 0.2533111	total: 17.5s	remaining: 9.41s
650:	learn: 0.2532908	total: 17.5s	remaining: 9.38s
651:	learn: 0.2532773	total: 17.5s	remaining: 9.36s
652:	learn: 0.2532230	total: 17.6s	remaining: 9.33s
653:	learn: 0.2531918	total: 17.6s	remaining: 9.3s
654:	learn: 0.2531629	total: 17.6s	remaining: 9.28s
655:	learn: 0.2531328	total: 17.6s	remaining: 9.25s
656:	learn: 0.2530940	total: 17.7s	remaining: 9.22s
657:	learn: 0.2530696	total: 17.7s	remaining: 9.2s
658:	learn: 0.2530443	total: 17.7s	remaining: 9.17s
659:	learn: 0.2530132	total: 17.7s	remaining: 9.14s
660:	learn: 0.2529859	total: 17.8s	remaining: 9.11s
661:	learn: 0.2529512	total: 17.8s	remaining: 9.08s
662:	learn: 0.2529071	total: 17.8s	remaining: 9.06s
663:	learn: 0.2528749	total: 17.8s	remaining: 9.03s
664:	learn: 0.

805:	learn: 0.2483041	total: 21.6s	remaining: 5.19s
806:	learn: 0.2482804	total: 21.6s	remaining: 5.17s
807:	learn: 0.2482666	total: 21.6s	remaining: 5.14s
808:	learn: 0.2482346	total: 21.7s	remaining: 5.11s
809:	learn: 0.2482092	total: 21.7s	remaining: 5.09s
810:	learn: 0.2481828	total: 21.7s	remaining: 5.06s
811:	learn: 0.2481617	total: 21.7s	remaining: 5.03s
812:	learn: 0.2481284	total: 21.8s	remaining: 5s
813:	learn: 0.2481084	total: 21.8s	remaining: 4.98s
814:	learn: 0.2480798	total: 21.8s	remaining: 4.95s
815:	learn: 0.2480600	total: 21.8s	remaining: 4.92s
816:	learn: 0.2480409	total: 21.9s	remaining: 4.9s
817:	learn: 0.2480087	total: 21.9s	remaining: 4.87s
818:	learn: 0.2479727	total: 21.9s	remaining: 4.84s
819:	learn: 0.2479477	total: 21.9s	remaining: 4.82s
820:	learn: 0.2479301	total: 22s	remaining: 4.79s
821:	learn: 0.2479037	total: 22s	remaining: 4.76s
822:	learn: 0.2478751	total: 22s	remaining: 4.74s
823:	learn: 0.2478600	total: 22s	remaining: 4.71s
824:	learn: 0.2478354	to

966:	learn: 0.2439251	total: 25.8s	remaining: 879ms
967:	learn: 0.2439091	total: 25.8s	remaining: 852ms
968:	learn: 0.2438849	total: 25.8s	remaining: 826ms
969:	learn: 0.2438612	total: 25.8s	remaining: 799ms
970:	learn: 0.2438398	total: 25.9s	remaining: 772ms
971:	learn: 0.2438045	total: 25.9s	remaining: 746ms
972:	learn: 0.2437844	total: 25.9s	remaining: 719ms
973:	learn: 0.2437671	total: 25.9s	remaining: 692ms
974:	learn: 0.2437497	total: 26s	remaining: 666ms
975:	learn: 0.2437329	total: 26s	remaining: 639ms
976:	learn: 0.2436665	total: 26s	remaining: 613ms
977:	learn: 0.2436392	total: 26s	remaining: 586ms
978:	learn: 0.2436188	total: 26.1s	remaining: 559ms
979:	learn: 0.2436045	total: 26.1s	remaining: 533ms
980:	learn: 0.2435613	total: 26.1s	remaining: 506ms
981:	learn: 0.2435202	total: 26.1s	remaining: 479ms
982:	learn: 0.2434998	total: 26.2s	remaining: 453ms
983:	learn: 0.2434750	total: 26.2s	remaining: 426ms
984:	learn: 0.2434487	total: 26.2s	remaining: 399ms
985:	learn: 0.243411

## Submission

In [26]:
submission = pd.DataFrame()
submission["employee_id"] = emp_id
submission["is_promoted"] = y_pred
submission.to_csv("sub_cat_final_2.csv", index=False)